## 1. 이터러블 vs 이터레이터 vs 제너레이터

1. iterable 이터러블
* ierable 객체 : 객체 안에 있는 원소(element)를 하나씩 반환 가능한 객체, iterator을 생성할 수 있는 객체
* 파이썬의 기본 내장 데이터 구조는 이터러블한 객체임. + 사용자가 만든 객체(class)도 iterable 객체가 될 수 있음.
- iterable 객체는 iter()라는 함수의 입력으로 들어감. 
    - iter()라는 함수는 다음의 iterator을 반환함.

2. iterator 이터레이터
* iterator 객체 : iterator의 __next__()나 내장함수인 next()를 호출하여 원소를 순차적으로 반환할 수 있는 객체
* iterator가 다음 원소를 계속 반환하다가 끝에 도달해 반활할 원소가 없는 경우 예외문인 StopIteration이 발생
- 파이썬에서 for문은 iterable 객체를 만나, 내부적으로 iter()함수를 호출하여 iterator 생성
    - 생성된 iterator는 루프가 실행되면서 next()를 호출해 반복적인 데이터를 반환 + 모든 원소 사용 후 StopIteration 발생하며 for문 종료

3. generator 제너레이터
* generator 객체 : iterator를 생성하는 함수. generator로 생성한 객체는 iterator와 마찬가지로 next()함수를 호출해 다음 값 얻기 가능.
* generator은 return 대신 yeild를 이용하여 값을 반환함.
    - return과 다르게 해당 함수(generator)가 종료되지 않고 그대로 유지 -> 다음 순서의 generator 호출 시 멈췄던 yield 자리에서 다시 함수 동작
    - generator -> next -> 다음 generator 실행 -> yield를 만나 값 반환(generator 종료 X) -> 다시 호출 -> 멈췄던 yield 부터 재실행
* 제너레이터는 변수 값을 지속적으로 바꾸어 리스트에 비해 메모리를 절약할 수 있는 장점을 가짐

#### 파이썬 iterator 내부 동작

``` python
# iter() 함수를 호출해, iterator 를 생성하고,
iterator_object = iter(iterable_object)

while True:
    # next() 함수를 호출해, element 를 받아옵니다.
    try:
        element = next(iterator_object)
        print(element)

    # element 가 없을 시, StopIteration Exception 발생
    except: StopIteration:
        break

```

In [2]:
# iterable 예시 
iterable_object = [1, 2, 3, 4, 5]
iterator_object = iter(iterable_object)
print(iterator_object)
for i in range(6):
    print(next(iterator_object))    # 호출 6번째에 StopIteration 발생

1
2
3
4
5


StopIteration: 

In [3]:
# iterable 클래스 예시
class MyIterable:

    def __init__(self, name_list, age_list):
        assert len(name_list) == len(age_list)  # name_list와 age_list 개수가 다르면 에러

        self.name_list = name_list
        self.age_list = age_list
        self._current = 0   # 리스트의 인덱스 변수
    
    def __iter__(self):
        return self

    def __next__(self):
        cur = self._current
        self._current += 1
        if self._current > len(self.name_list): # 리스트 범위 벗어날 시 종료
            print("!!") # if문 조건을 만족하고 종료
            raise StopIteration
        return (self.name_list[cur], self.age_list[cur])

for element in MyIterable(['test1', 'test2', 'test3'], [15, 19, 23]):
    print(element)

('test1', 15)
('test2', 19)
('test3', 23)
!!


In [4]:
def generator_square():
    for number in range(0, 5):
        yield number**2

square_result = generator_square()  # 제너레이터 객체 생성
square_result2 = generator_square() # 제너레이터 객체2 생성
print(next(square_result))
print(next(square_result))
print(next(square_result2))         # result와 result2는 서로 다른 객체
print(next(square_result2))

0
1
0
1


In [5]:
# generator의 장점 비교

# os.getpid() -> 파이썬 프로세스 ID 가져오기
# psutil.Process(os.getpid()) -> psutil 모듈로 PID를 나타내는 Process 객체 생성 (psutil: 시스템과 프로세스 정보 모니터링 라이브러리)
# <Process>.memort_info().rss -> 현재 프로세스의 메모리 정보 가져오기.(rss: Resident Set size, 현재 프로세스가 사용중인 메모리 크기(단위:Byte))

# for문으로 0~999999 범위의 각 숫자를 제곱값을 리스트에 저장해 반환하는 코드 
# 메모리 사용 위치 (단위:MB) 시작 주소 : 69.47265625 -> 끝 주소: 108.75

import os
import psutil

def non_generator_square(end):
    result = []
    for number in range(0, end):
        result.append(number**2)

    return result

mem_usage_start = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)
square_results = non_generator_square(999999)
mem_usage_end = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

print("Memory Usage when program start: {}".format(mem_usage_start))
print("Memory Usage when program end: {}".format(mem_usage_end))

Memory Usage when program start: 72.1875
Memory Usage when program end: 111.4375


In [6]:
# generator로 0~999999 범위의 각 숫자를 제곱값을 반환하는 코드 
# 메모리 사용 위치 (단위:MB) 시작 주소 : 69.265625 -> 끝 주소 : 69.265625

import os
import psutil

def generator_square(end):
    for number in range(0, end):
        yield number**2

mem_usage_start = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)
square_results = generator_square(999999)
mem_usage_end = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

print("Memory Usage when program start: {}".format(mem_usage_start))
print("Memory Usage when program end: {}".format(mem_usage_end))        

Memory Usage when program start: 111.4375
Memory Usage when program end: 73.84375


## 2. 어노테이션(annotation)

* 파이썬은 다른 언어와 달리 변수에 명시적으로 자료형을 명시하지 않음 -> 이를 보완하기 위해 annotation 사용
* 변수의 예상 타입을 명시하는데 사용 (변수의 실제 타입을 강제 X)
- 코드의 가독성 증가
- 정적 타입 검사 도구(mypy)를 사용해 타입 오류를 사전에 잡기 위한 목적으로 사용

1. 기본 자료형 annotation
    - 주로 변수나 자룟값의 변수뒤에 :와 = 명시함으로써, 자료형을 메모

In [7]:
name : str = 'peter'    # name = 'peter'
age: int = 15   # age = 15
height: float=165.5 # height = 165.5
a: bool=False   # a = False

print(name)
print(type(name))

peter
<class 'str'>


2. 컬렉션 자료형 어노테이션
    - (리스트, 튜플, 딕셔너리, 세트)에 대한 어노테이션 표현방법은 기본보다 복잡
    - 이 자료형들은 typing 모듈에서 가져올 수 있음  ->   from typing import List,Tuple,Dict,Set
    - 어노테이션 표현 시, [] 사용

In [8]:
from typing import List,Tuple,Dict,Set

#리스트 어노테이션
a: List[str]=['one','two','three']  # a=['one','two','three']

#튜플 어노테이션
b:Tuple[int]=(1,2,3)                # b=(1,2,3)

#딕셔너리 어노테이션
c:Dict[str,int]={'one':1,'two':2,'three':3} # c={'one':1,'two':2,'three':3}

#셋 어노테이션      set : 집합 자료형 (중복 허용 X, 순서 X)
d:Set[int]={1,2,3}                  # d={1,2,3}

print(a)
print(b)
print(c)
print(d)

['one', 'two', 'three']
(1, 2, 3)
{'one': 1, 'two': 2, 'three': 3}
{1, 2, 3}


3. 함수 어노테이션
    - 사용자 정의 함수를 만들 때, 함수안의 변수들을 어노테이션으로 정의 가능
    - 함수 어노테이션 확인 : <함수>.\_\_annotations\_\_
    

In [9]:
#함수 어노테이션

def add(a: int, b:int)->int:    # int인 입력 a,b를 int로 반환
    return a+b

print(add(7,5))
print(add('칠', '오'))

#함수의 어노테이션 확인
add.__annotations__

12
칠오


{'a': int, 'b': int, 'return': int}

4. 클래스 어노테이션

In [10]:
class Student:
    def __init__(self, name:str, age:int):
        self.name = name
        self.age = age
    def __str__(self) -> str:   # __str__ 을 오버라이딩하여 print()의 출력 값을 메모리 주소 -> 문자열 출력으로 변경
        return '이름: ' + self.name + ', 나이:' + str(self.age)

def a(s:Student) -> None:   
    print(f'이름: {s.name}, 나이:{s.age} 🍖')

student=Student('mksd', 23)    # Student 객체 생성
print(student)  # __str__ 호출
a(student)      # 함수 a 호출

이름: mksd, 나이:23
이름: mksd, 나이:23 🍖


5. 옵셔널
    - 변수가 특정 타입이거나 변수의 메모리 값이 None일 때 사용
    - Optional[x] -> 반환 값이 x 이거나 None 인 것을 명시

In [11]:
from typing import Optional, Dict     #옵셔널 모듈 사용

#함수 지정
def find_score(name: str, scores: Dict[str,float])->Optional[float]:        # 반환 값이 float형 또는 None임을 명시
    return scores.get(name)

print(find_score('김사과', {'김사과': 80.5, '오렌지':40.8, '반하나':90.4}))     # 반환 자료형 : float
print(find_score('파인애플', {'김사과': 80.5, '오렌지':40.8, '반하나':90.4}))   # 반환 자료형 : None

80.5
None
